In [47]:
from  snippets.reader  import *
from snippets.transform import *
from snippets.vectorization import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
%matplotlib inline

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [48]:
# 日本語に対応しているフォントを以下の中から
# import matplotlib.font_manager
# print([f.name for f in matplotlib.font_manager.fontManager.ttflist])
# で出力されるものの中から選ぶ
mpl.rcParams['font.family'] = 'AppleGothic'


plt.rcParams["figure.figsize"] = [20, 15] # グラフのサイズを指定
plt.rcParams['font.size'] = 20 #フォントサイズを設定 default : 12
plt.rcParams['xtick.labelsize'] = 25 # 横軸のフォントサイズ
plt.rcParams['ytick.labelsize'] = 25 # 縦軸のフォントサイズ
plt.rcParams['legend.fontsize'] = 25 # 凡例のフォントサイズ
plt.rcParams['axes.titlesize'] = 30 # タイトルのフォントサイズ
plt.rcParams['axes.labelsize'] = 25 # 軸ラベルのフォントサイズ

# テキストの読み込み

In [49]:
natume=JapaneseCorpusReader('./dataset/corpas/natsume/',encoding='shift_jis')
dazai=JapaneseCorpusReader('./dataset/corpas/dazai/',encoding='shift_jis')

# 規格化

In [ ]:
nomalizer=JapaneseTextNormalizer()
natume_normed=nomalizer.transform(natume,['助詞', '助動詞', '記号','動詞'])
dazai_normed=nomalizer.transform(dazai,['助詞', '助動詞', '記号','動詞'])

In [ ]:
# 正解ラベルを作成
target=np.hstack((np.zeros(len(natume_normed)),np.ones(len(dazai_normed))))
target[:len(natume_normed)+1]

In [ ]:
len(target)

# 可視化

### 頻度分布

In [ ]:
natume_word=pd.Series(sum(natume_normed,[]))
dazai_word=pd.Series(sum(dazai_normed,[]))

In [ ]:
natume_count=natume_word.value_counts()
natume_count.head(20)

In [ ]:
dazai_count=dazai_word.value_counts()
dazai_count.head(20)

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2)
natume_count.head(10).plot.bar(ax=ax1,rot=60,label="漱石",title='漱石作品')
dazai_count.head(10).plot.bar(ax=ax2,rot=60,label="太宰",title="太宰作品")

## それぞれの作品に特徴的な言葉を見つける

In [ ]:
dazai_word_set=set(dazai_word)
natume_word_set=set(natume_word)


### 太宰しかでてない言葉の頻度分布


In [ ]:
# 差集合を取る
only_dazai=dazai_word_set.difference(natume_word_set)
only_dazai_count=dazai_count.loc[only_dazai].sort_values(ascending=False)


## 夏目漱石しか出ていない言葉の頻度分布

In [ ]:
# 差集合を取る
only_natume=natume_word_set.difference(dazai_word_set)
only_natume_count=natume_count.loc[only_natume].sort_values(ascending=False)


In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2)
only_natume_count.head(10).plot.bar(ax=ax1,rot=60,label="漱石",title='漱石作品にのみ出ることば')
only_dazai_count.head(10).plot.bar(ax=ax2,rot=60,label="太宰",title="太宰作品のみに出ることば")

# ベクトル化

## tf_idf


In [ ]:
tfidf=TfidfVectorizer()

In [ ]:
## 全データを結合
all_data=dazai_normed+natume_normed
# 変換
all_data=tfidf.transform(all_data)
all_data=pd.DataFrame(all_data).fillna(0)
all_data=all_data.to_sparse(fill_value=0)


In [ ]:
all_data.head()

##  targetを作成

In [ ]:
target_df=pd.Series(target)
target_df.name='target'

# モデルに適用

## データの分割

In [ ]:
x_train,x_test,t_train,t_test=train_test_split(all_data,target_df,random_state=1)

In [ ]:
model=LogisticRegression()
model.fit(x_train,t_train)
pred=model.predict(x_test)

# 評価

In [ ]:
print(classification_report(t_test,pred))